In [8]:
# !pip install --upgrade google-cloud-bigquery-storage

In [9]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import storage, bigquery

project_id = 'wpac-449904'
# Create a storage client
client = bigquery.Client(project=project_id)

In [10]:
query = """
SELECT * FROM `wpac-449904.wpac_data.all_time_connect`
"""
df = client.query(query).to_dataframe()

# Display first few rows
df.head()

,_id,collectionType,eventName,createdAt,partnerRegion,sessionId,data_careerId,data_jobId,data_programId,data_resourceId,WPA_id
0,6663e9a6daf5f872d641dce5,NA,edited status,2024-06-08 09:18:30+00:00,Atlanta,e059e07d-48c1-49e4-af6e-162add85fc9b,NA,NA,630c6a622c7386c831f3d16c,NA,WPA_105333
1,6648b3e7bbe71cd48bb64f6c,NA,edited status,2024-05-18 17:57:59+00:00,Atlanta,fce6d678-5e24-4357-aa02-15a87f4e2f97,NA,NA,64c976148776a05127ca769e,NA,WPA_106152
2,6648dec733dcbb5ec5de8904,NA,edited status,2024-05-18 21:00:58+00:00,Atlanta,3ba85c3e-d4b1-4483-8412-2879a85a3fa6,NA,NA,64c9772a8776a05127ca867d,NA,WPA_106199
3,6648dec9bbe71cd48bb6cc74,NA,edited status,2024-05-18 21:01:00+00:00,Atlanta,3ba85c3e-d4b1-4483-8412-2879a85a3fa6,NA,NA,64c9772a8776a05127ca867d,NA,WPA_106199
4,6678b884c8205e4485c657b3,NA,edited status,2024-06-24 04:06:27+00:00,Atlanta,fd45e415-3a71-4034-9856-a77045e05329,NA,NA,628aed8312545d84f96e82c7,NA,WPA_106392


In [11]:
df.isnull().sum()

,0
_id,0
collectionType,0
eventName,0
createdAt,0
partnerRegion,0
sessionId,0
data_careerId,0
data_jobId,0
data_programId,0
data_resourceId,0


In [12]:
print(df.shape)

(895858, 11)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895858 entries, 0 to 895857
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   _id              895858 non-null  object             
 1   collectionType   895858 non-null  object             
 2   eventName        895858 non-null  object             
 3   createdAt        895858 non-null  datetime64[us, UTC]
 4   partnerRegion    895858 non-null  object             
 5   sessionId        895858 non-null  object             
 6   data_careerId    895858 non-null  object             
 7   data_jobId       895858 non-null  object             
 8   data_programId   895858 non-null  object             
 9   data_resourceId  895858 non-null  object             
 10  WPA_id           895858 non-null  object             
dtypes: datetime64[us, UTC](1), object(10)
memory usage: 75.2+ MB


In [14]:
print(df.nunique())

_id                895858
collectionType          5
eventName               5
createdAt          877581
partnerRegion          31
sessionId          133446
data_careerId          64
data_jobId          57377
data_programId       4564
data_resourceId       235
WPA_id             132155
dtype: int64


In [15]:
dataset_id = "wpac_data"

client = bigquery.Client(project=project_id)
datasets = [dataset.dataset_id for dataset in client.list_datasets()]
print("Available Datasets:", datasets)

if dataset_id not in datasets:
    print(f"Dataset {dataset_id} does not exist. Creating it now...")
    dataset_ref = client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Change if necessary
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created.")


Available Datasets: ['wpac_data']


In [16]:
import pandas_gbq

dataset_table = "wpac_data.cleaned_all_time_connect"  # Replace with dataset.table name

# Load your CSV into a DataFrame
csv_file_path = "cleaned_all_time_connect.csv"  # Replace with your actual file path
# df = pd.read_csv(csv_file_path)

# Use `pandas_gbq.to_gbq` to upload DataFrame
pandas_gbq.to_gbq(
    df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists="replace"  # "fail", "replace", "append"
)


100%|██████████| 1/1 [00:00<00:00, 7612.17it/s]
